#  Juliaで高速なベクトル・行列の区間演算をしたい

ベクトルの内積、行列ベクトル積、行列積の区間演算はとても重要。すなわち
$x, y\in\mathbb{IF}^n$, $A, B\in\mathbb{IF}^{n\times n}$ に対して、それぞれ$x^Ty \subset \alpha$, $Ax\subset z$, $AB\subset C$となる
$$
\alpha\in\mathbb{IF},\quad z\in\mathbb{IF}^n,\quad C\in\mathbb{IF}^{n\times n}
$$
を求める方法を考える。

## 素朴な方法

最も素朴な方法は各演算を区間演算に変更した方式である。

$$
    \alpha = \sum_{k = 1}^n x_ky_k
$$

$$
    z_i = \sum_{k = 1}^n A_{i,k}x_k\quad (1\le i\le n)
$$

$$
    C_{ij} = \sum_{k+1}^n A_{i,k}B_{k,j}\quad (1\le i,j\le n)
$$

特にJuliaの`IntervalArithmetic.jl`パッケージでは、この方法で区間演算が行なわれている。しかし、この方法だとプログラムの最適化が難しく、計算機の性能を十分に引き出した実装が難しく、計算時間がかかってしまう。

In [3]:
using IntervalArithmetic, BenchmarkTools

n = 1000;
x = randn(n);
y = randn(n);

x_int = map(Interval, x);
y_int = map(Interval, y);

@btime alpha = dot(x_int,y_int)

# setrounding(Interval, :fast)

# function dot_vec()
#     n = 1000; x = randn(n); y = randn(n);
#     x_int = map(Interval, x); y_int = map(Interval, y);
#     alpha = dot(x_int,y_int);
# end
# @btime dot_vec()

  69.187 μs (6007 allocations: 93.88 KiB)


In [5]:
using LinearAlgebra
n = 1000; A = randn(n,n); x = randn(n); 
A_int = map(Interval, A); x_int = map(Interval, x); y = similar(x_int);
@time z = A_int*x_int;
@time mul!(y,A_int,x_int)
println(y ⊆ z)
println(z ⊆ y)

# @test z ⊆ y
# @btime z = A_int*x_int;

  0.046672 seconds (1 allocation: 15.750 KiB)
  0.045570 seconds
true
true


In [8]:
n = 100;
A = randn(n,n);
B = randn(n,n);
A_int = map(Interval, A);
B_int = map(Interval, B);
@time C = A_int*B_int;
@time C = A_int*B_int;
@time C = A_int*B_int;

  0.048638 seconds (8 allocations: 156.656 KiB)
  0.046084 seconds (8 allocations: 156.656 KiB)
  0.046330 seconds (8 allocations: 156.656 KiB)


In [41]:
n = 1000; A = randn(n,n); B = randn(n,n);
A_int = map(Interval, A); B_int = map(Interval, B);
@btime C = A_int*B_int;

  41.813 s (8 allocations: 15.26 MiB)


これを高速化したい。考えられる方法は

1. 並列化したい（[Loopvectrization](https://github.com/chriselrod/LoopVectorization.jl), [Tullio](https://github.com/mcabbott/Tullio.jl)?, @simd命令）
1. [StaticArrays](https://github.com/JuliaArrays/StaticArrays.jl_)を使うと[速いらしい](https://qiita.com/cometscome_phys/items/669273a49ab2417d3af8)

一つ目はまだ未実験、二つ目は小さいサイズでは多少計算時間が変わるが、要素数が多くなるほどコンパイルに時間がかかり、全体の経過時間は長くなってしまった。要素数が100以下なら`StaticArrays`が使えると上記Webサイトに書いてありました。

そこで残るは

3. BLASを使う区間演算を実装する、だけど丸め方向が変えられないので、丸めの向きを変えない区間演算
    - 内積は森倉定理8 (区間ベクトルの内積はいる？)
    - 行列ベクトル積はいる？
    - 行列積は森倉4節
 
 
 ## BLASを使う
 
 まずJuliaの中で、[BLAS](https://ja.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms)を使うには``LinearAlgebra``パッケージが必要。

In [9]:
using LinearAlgebra, BenchmarkTools
# versioninfo()
# BLAS.vendor()
n = 5000;
A, B, C = randn(n,n), randn(n,n), zeros(n,n);

@btime C = A*B;
@btime mul!($C, $A, $B);

  755.193 ms (2 allocations: 190.73 MiB)
  816.399 ms (0 allocations: 0 bytes)


このようにBLASを使うと、物凄いスピードで数値計算ができる。ただし、行列の内部の型がfloat64に限られるのと、丸め方向を変更した計算ができないため、区間演算の結果が粗くなる（区間幅が増加する）。

**定義**　$\mathbf{u}=2^{-53}$を倍精度の**単位相対丸め**とする。$\mathbf{S}_{\min}=2^{-1074}$を倍精度浮動小数点数の正の最小数とする。$\mathbf{F}_{\min}=2^{-1022}$を倍精度浮動小数点数の正規化数の正の最小数とする。 

**注意**　単位相対丸めは$1$と$1$よりも小さい最大の浮動小数点数との差を表す。つまり
$$
1-\mathbf{u}<a<1
$$
となる$a\in\mathbb{F}$は存在しない。

**注意**　単位相対丸めとは別に**計算機イプシロン**（unit roundoff, machine epsilon）という単位もあり、こちらは$1$と$1$よりも大きい最小の浮動小数点数との差を表し、64bit浮動小数点数では$2^{-52}$でこちらを$\mathbf{u}$と書く流儀もある（こっちが主流？）ので、注意が必要。Juliaでは`eps(Float64)`とすると数値が得られる。

In [2]:
u = 2.0^-53;
s_min = 2.0^-1074;
f_min = 2.0^-1022;
println(bitstring(1.0))
println(bitstring(1.0-u))
println(bitstring(s_min))
println(bitstring(f_min))
println(eps(Float64))
println(2*u)

0011111111110000000000000000000000000000000000000000000000000000
0011111111101111111111111111111111111111111111111111111111111111
0000000000000000000000000000000000000000000000000000000000000001
0000000000010000000000000000000000000000000000000000000000000000
2.220446049250313e-16
2.220446049250313e-16


**定義**　実数$a\in\mathbb{R}$に対する**U**nit in the **F**irst **P**lace (ufp)は次のように定める。

$$
 \mbox{ufp}(a):= 2^{\lfloor\log_2|a|\rfloor} (a\neq 0),\quad \mbox{ufp}(0)=0.
$$

関数$\mbox{ufp}$は実数$a$を2進数表記した先頭ビットを返す。4回の浮動小数点演算で求めることができる。

**アルゴリズム**

```
function s = ufp(p)
    phi = (2*u)^{-1} + 1;
    q = phi * p;
    s = |q - (1-u)q|;
end
```

関数$\mbox{ufp}$を使うと$\mbox{ufp}(a)\le a < 2\mbox{ufp}$が成立する。

In [3]:
function ufp(P)
    u = 2.0^(-53);
    ϕ = 2.0^52 + 1;
    q = ϕ * P;
    T = (1 - u)*q;
    return abs(q - T);
end

ufp (generic function with 1 method)

In [10]:
p = 1.0 - u;
println(bitstring(p))
println(bitstring(ufp(p)))
println(bitstring(2*ufp(p)))

0011111111101111111111111111111111111111111111111111111111111111
0011111111100000000000000000000000000000000000000000000000000000
0011111111110000000000000000000000000000000000000000000000000000


**定義**　$c\in\mathbb{R}$として、

- $\mbox{pred}$: $r$より小さい最大の浮動小数点数を返す関数, $\mbox{pred}(c):=\max\{f\in \mathbb{F}:f<c\}$
- $\mbox{succ}$: $r$より大きい最小の浮動小数点数を返す関数, $\mbox{succ}(c):=\min\{f\in \mathbb{F}:c<f\}$


これらを使うと$a$, $b\in\mathbb{F}$, $\circ\in\{+,-,\times,\div\}$で
$$
    \mbox{pred}(\mbox{fl}(a\circ b))<a\circ b<\mbox{succ}(\mbox{fl}(a\circ b))
$$
が成り立つ。

また、pred関数, succ関数はベクトル・行列においても各要素に対するsucc, predを考える事で拡張することが出来る。

In [19]:
function succ(c)
    s_min = 2.0^-1074;
    u = 2.0^-53;
    ϕ = u * (1.0 + 2.0 * u);
    if abs(c) >= (1. / 2.) * u^(-2) * s_min # 2^(-969)(Float64)
        e = ϕ * abs(c);
        succ = c + e;
    elseif abs(c) < (1. / 2.) * u^(-1) * s_min # 2^(-1022)(Float64)
        succ = c + s_min;
    else
        C = u^(-1) * c;
        e = ϕ * abs(C);
        succ = (C + e) * u;
    end
    return succ
end

function pred(c)
    s_min = 2.0^-1074;
    u = 2.0^-53;
    ϕ = u * (1.0 + 2.0 * u);
    if abs(c) >= (1. / 2.) * u^(-2) * s_min # 2^(-969)(Float64)
        e = ϕ * abs(c);
        pred = c - e;
    elseif abs(c) < (1. / 2.) * u^(-1) * s_min # 2^(-1022)(Float64)
        pred = c - s_min;
    else
        C = u^(-1) * c;
        e = ϕ * abs(C);
        pred = (C - e) * u;
    end
    return pred
end

pred (generic function with 1 method)

In [24]:
a = 0.1;
println(bitstring(succ(a)))
println(bitstring(a))
println(bitstring(pred(a)))

0011111110111001100110011001100110011001100110011001100110011011
0011111110111001100110011001100110011001100110011001100110011010
0011111110111001100110011001100110011001100110011001100110011001


In [125]:
a = randn(3,3);
abs.(a)


3×3 Array{Float64,2}:
 0.124605  1.55775   1.27203
 0.505403  1.08982   0.901059
 0.828683  0.396004  1.30595

In [163]:
function mm_ufp(A_mid, A_rad, B_mid, B_rad)
    u = 2.0^(-53);
    realmin = 2.0^(-1022);
    n = size(A_mid,2);
    
    if(2*(n+2)*u>=1)
        error("mm_ufp is failed!(2(n+2)u>=1)")
    end
    C_mid = A_mid * B_mid;
    C_rad = (n+2) * u * ufp.(abs.(A_mid)*abs.(B_mid)) .+ realmin;
    
    AmBr = abs.(A_mid) * B_rad;
    AmBr = succ.(AmBr + ((n+2)*u*ufp.(AmBr) .+ realmin));
    
    ArBm = A_rad * abs.(B_mid);
    ArBm = succ.(ArBm + ((n+2)*u*ufp.(ArBm) .+ realmin));

    ArBr = A_rad * B_rad;
    ArBr = succ.(ArBr + ((n+2)*u*ufp.(ArBr) .+ realmin));

    rad_sum = C_rad + AmBr + ArBm + ArBr;

    C_rad = succ.(rad_sum + 3*u*ufp.(rad_sum));
    
    return C_mid, C_rad;
end

mm_ufp (generic function with 1 method)

In [191]:
using IntervalArithmetic, LinearAlgebra

n = 3000;
A = randn(n,n);
B = randn(n,n);
A_int = map(Interval, A);
B_int = map(Interval, B);
# @time C = A_int*B_int;

A_mid = mid.(A_int);
A_rad = radius.(A_int);
B_mid = mid.(B_int);
B_rad = radius.(B_int);
C_mid = similar(A_mid)
C_rad = similar(A_rad);
@time C_mid, C_rad = mm_ufp(A_mid,A_rad,B_mid,B_rad);
C_int = C_mid .± C_rad;

  1.665447 seconds (67 allocations: 2.146 GiB, 8.05% gc time)


In [187]:
# C .⊂ C_int
println(maximum(radius.(C[:])))
println(maximum(C_rad[:]))

7.815970093361102e-14
7.247535904753025e-13


## 参考文献
1. S. M. Rump, P. Zimmermann, S. Boldo and G. Melquiond: “Computing predecessor and successor in rounding to nearest”, BIT Vol. 49, No. 2, pp.419–431, 2009.
(http://www.ti3.tu-harburg.de/paper/rump/RuZiBoMe08.pdf)
1. 柏木雅英, IEEE754 浮動小数点数の隣の数を計算する方法のまとめ, [http://www.kashi.info.waseda.ac.jp/~kashi/lec2020/nac/succ.pdf](http://www.kashi.info.waseda.ac.jp/~kashi/lec2020/nac/succ.pdf)